<a href="https://colab.research.google.com/github/Darktechie-programmer/Deep_learning/blob/master/Hyper_parameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### day 6: 9th may, 2020   lect- 18
##### Topic : Hyper parameter Tuning To Decide Number of Hidden Layers in Neural Network 

In [0]:
# note : In this lect we know about how to decide how many Hidden layers and hidden neuron are used...

### Artificial Neural Network

In [0]:
# part 1 - Data Preprocessing

In [0]:
# importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
# Importing the dataset
dataset = pd.read_csv("sample_data/Churn_Modelling.csv")
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
dataset.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [6]:
X = dataset.iloc[:,3:13]
y = dataset.iloc[:,-1]
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [0]:
# Create Dummy variables...
geography = pd.get_dummies(X["Geography"],drop_first=True)
gender = pd.get_dummies(X['Gender'],drop_first=True) # drop_first : it drop first dummy for multicoleanerity reason....

In [8]:
# Concatenate the Data Frames
X = pd.concat([X,geography,gender],axis=1)
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,France,Female,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,1,0


In [0]:
# Drop Unnecessary columns
X = X.drop(columns=['Geography','Gender'],axis=1)

In [10]:
X.head()
# X.columns

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [0]:
# Spliting dataset into traing set to testing set....
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [0]:
# Feature Scaling 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [0]:
## Perform Hyperparametric Optimization

In [14]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
# note: we can also use the randomSearchCV for same work 
# note: GridSearchCV is used for selecting right kind of hyper meters that is calling hyper_meter tuning...

Using TensorFlow backend.


In [0]:
## import necessary libraries...
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid

In [0]:
## create a zenric code to check that How many hidden layers required....
def create_model(layers, activation):
  model = Sequential()
  for i, nodes in enumerate(layers):
    if i==0:
      model.add(Dense(nodes, input_dim=X_train.shape[1]))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
    else:
      model.add(Dense(nodes))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
  model.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))
  # note: If we used relu activation then use he_normal or he_uniform.. but in the case when we use sigmoid activation function then we use glorot_unifrom or glorot_normal in kernel_initializer.....
  
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [18]:
model = KerasClassifier(build_fn=create_model, verbose=0)
model 

In [0]:
# layers=[[20],[40,20],[45,30,15]] Note Error here.... so for fix
layers=[(20),(40,20),(45,30,15)]
activations = ['sigmoid', 'relu']
param_grid= dict(layers=layers, activation=activations, batch_size=[128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)

### By stack OverFlow

I had the same problem. It seems to be a bug in keras (https://github.com/keras-team/keras/issues/13586) that occurs with nested arrays as parameters for the grid search. I was able to solve it by nesting tuples instead of arrays.

So try to change

layers = [[50],[50, 20], [50, 30, 15], [70,45,15,5]]

to

layers = [(50),(50, 20), (50, 30, 15), (70,45,15,5)]

In [20]:
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: 'int' object is not iterable

  FitFailedWarning)


In [0]:
## model best Solution

In [22]:
print(grid_result.best_score_, grid_result.best_params_)

0.8536249995231628 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': (45, 30, 15)}


In [23]:
pred_y = grid.predict(X_test)
y_pred = (pred_y > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [24]:
pred_y

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [0]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [26]:
confusion_matrix(y_pred, y_test)

array([[1528,  206],
       [  67,  199]])

In [27]:
accuracy_score(y_pred, y_test)

0.8635